In [1]:
!pip install torchfile
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
import pandas as pd
import torchfile

# from facenet_pytorch import InceptionResnetV1, fixed_image_standardization, training
# a = []
# while(1):
#     a.append(1)

  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5712 sha256=bb179a1266412939af0b29a9543ee755ed07ee699020e1372c9df573f88b2a19
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


In [2]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%%time
zip_path = '/content/drive/My Drive/Colab Notebooks/allimgs_face2.zip'
!cp "{zip_path}" .
!unzip -q allimgs_face2.zip 
!rm allimgs_face2.zip

CPU times: user 53.7 ms, sys: 27.6 ms, total: 81.3 ms
Wall time: 19.2 s


In [4]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_pairs_race_split.csv')

winners_train = data[[data.columns[1], data.columns[2]]]
losers_train = data[[data.columns[3], data.columns[4]]]

test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test_pairs_race_split.csv')
winners_test = test_data[[test_data.columns[1], test_data.columns[2]]]
losers_test = test_data[[test_data.columns[3], test_data.columns[4]]]

val_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/val_pairs_race_split.csv')
winners_val = val_data[[val_data.columns[1], val_data.columns[2]]]
losers_val = val_data[[val_data.columns[3], val_data.columns[4]]]

In [5]:
# # Get train and test data for classification
train_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_on_cand.csv')
new_paths = []
for path, winner in zip(train_df[train_df.columns[2]].tolist(), train_df[train_df.columns[3]].tolist()):
    words = path.split('/')
    new_path = 'allimgs_face/'+words[1]+'/'+words[2]
    if '.webp' not in new_path:
        new_paths.append([new_path, winner])
train_df = pd.DataFrame(new_paths) 
train_df[train_df.columns[1]]= train_df[train_df.columns[1]].astype(float)
# train_df = train_df.sample(frac=1).reset_index(drop=True)


test_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test_on_cand.csv')
new_paths_test = []
for path, winner in zip(test_df[test_df.columns[2]].tolist(), test_df[test_df.columns[3]].tolist()):
    words = path.split('/')
    new_path = 'allimgs_face/'+words[1]+'/'+words[2]
    if '.webp' not in new_path:
        new_paths_test.append([new_path, winner])

test_df = pd.DataFrame(new_paths_test)    
test_df[test_df.columns[1]]= test_df[test_df.columns[1]].astype(float)   

val_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/val_on_cand.csv')
new_paths_val = []
for path, winner in zip(val_df[val_df.columns[2]].tolist(), val_df[val_df.columns[3]].tolist()):
    words = path.split('/')
    new_path = 'allimgs_face/'+words[1]+'/'+words[2]
    if '.webp' not in new_path:
        new_paths_val.append([new_path, winner])


val_df = pd.DataFrame(new_paths_val)    
val_df[val_df.columns[1]]= val_df[val_df.columns[1]].astype(float)   
# val_df

In [6]:
# Create a dataloader class
class DatasetFaces_Rank(Dataset):
    
    def __init__(self, df, transform=None):
        self.data = df
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):

        img_name = self.data[self.data.columns[0]].iloc[index]
        # img_name = img_name.split('/')

   

        image = Image.open("/content/"+img_name).convert('RGB')
        # label = self.data[self.data.columns[1]].iloc[index]
        if self.transform is not None:
            image = self.transform(image)
        else:
            print("TRANSFORM FAILED")
        return image
  
# Create a dataloader class
class DatasetFaces_classify(Dataset):
    
    def __init__(self, df, transform=None):
        self.data = df
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):

        img_name = self.data[self.data.columns[0]].iloc[index]
        # img_name = img_name.split('/')

   

        image = Image.open("/content/"+img_name).convert('RGB')
        label = self.data[self.data.columns[1]].iloc[index]
        if self.transform is not None:
            image = self.transform(image)
        else:
            print("TRANSFORM FAILED")
        return image,label

In [7]:
# Create a transform for data prep
transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([123.68, 116.78, 103.94], [1, 1, 1]),
    ])
# Ranking
extraction_dataset_win = DatasetFaces_Rank(winners_train, transform)
extraction_loader_win = torch.utils.data.DataLoader(extraction_dataset_win, batch_size=1,shuffle=False, num_workers=1)
extraction_dataset_los = DatasetFaces_Rank(losers_train, transform)
extraction_loader_los = torch.utils.data.DataLoader(extraction_dataset_los, batch_size=1,shuffle=False, num_workers=1)

extraction_dataset_win_test = DatasetFaces_Rank(winners_test, transform)
extraction_loader_win_test = torch.utils.data.DataLoader(extraction_dataset_win_test, batch_size=1,shuffle=False, num_workers=1)
extraction_dataset_los_test = DatasetFaces_Rank(losers_test, transform)
extraction_loader_los_test = torch.utils.data.DataLoader(extraction_dataset_los_test, batch_size=1,shuffle=False, num_workers=1)

extraction_dataset_win_val = DatasetFaces_Rank(winners_val, transform)
extraction_loader_win_val = torch.utils.data.DataLoader(extraction_dataset_win_val, batch_size=1,shuffle=False, num_workers=1)
extraction_dataset_los_val = DatasetFaces_Rank(losers_val, transform)
extraction_loader_los_val = torch.utils.data.DataLoader(extraction_dataset_los_val, batch_size=1,shuffle=False, num_workers=1)

# Classification
train_dataset = DatasetFaces_classify(train_df, transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1,shuffle=False, num_workers=1)

test_dataset = DatasetFaces_classify(test_df, transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,shuffle=False, num_workers=1)


val_dataset = DatasetFaces_classify(val_df, transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1,shuffle=False, num_workers=1)

In [8]:
class VGG_16(nn.Module):
    """
    Main Class
    """

    def __init__(self):
        """
        Constructor
        """
        super().__init__()
        self.block_size = [2, 2, 3, 3, 3]
        self.conv_1_1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.conv_1_2 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        self.conv_2_1 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.conv_2_2 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        self.conv_3_1 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.conv_3_2 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.conv_3_3 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.conv_4_1 = nn.Conv2d(256, 512, 3, stride=1, padding=1)
        self.conv_4_2 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv_4_3 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv_5_1 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv_5_2 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.conv_5_3 = nn.Conv2d(512, 512, 3, stride=1, padding=1)
        self.fc6 = nn.Linear(512 * 7 * 7, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 2622)

    def load_weights(self, path="/content/drive/My Drive/Colab Notebooks/VGG_FACE.t7"):
        """ Function to load luatorch pretrained
        Args:
            path: path for the luatorch pretrained
        """
        model = torchfile.load(path)
        counter = 1
        block = 1
        for i, layer in enumerate(model.modules):
            if layer.weight is not None:
                if block <= 5:
                    self_layer = getattr(self, "conv_%d_%d" % (block, counter))
                    counter += 1
                    if counter > self.block_size[block - 1]:
                        counter = 1
                        block += 1
                    self_layer.weight.data[...] = torch.tensor(layer.weight).view_as(self_layer.weight)[...]
                    self_layer.bias.data[...] = torch.tensor(layer.bias).view_as(self_layer.bias)[...]
                else:
                    self_layer = getattr(self, "fc%d" % (block))
                    block += 1
                    self_layer.weight.data[...] = torch.tensor(layer.weight).view_as(self_layer.weight)[...]
                    self_layer.bias.data[...] = torch.tensor(layer.bias).view_as(self_layer.bias)[...]

    def forward(self, x):
        """ Pytorch forward
        Args:
            x: input image (224x224)
        Returns: class logits
        """
        x = F.relu(self.conv_1_1(x))
        x = F.relu(self.conv_1_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_2_1(x))
        x = F.relu(self.conv_2_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_3_1(x))
        x = F.relu(self.conv_3_2(x))
        x = F.relu(self.conv_3_3(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_4_1(x))
        x = F.relu(self.conv_4_2(x))
        x = F.relu(self.conv_4_3(x))
        x = F.max_pool2d(x, 2, 2)
        x_5_1 = F.relu(self.conv_5_1(x))
        x_5_2 = F.relu(self.conv_5_2(x_5_1))
        x_5_3 = F.relu(self.conv_5_3(x_5_2))
        x = F.max_pool2d(x_5_3, 2, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, self.training)
        x = F.relu(self.fc7(x))
        x = F.dropout(x, 0.5, self.training)
        # return self.fc8(x)
        return(x_5_1, x_5_2,x_5_3)

In [9]:
# Load model and create model
model = VGG_16()
model.load_weights()
model.fc8 = nn.Linear(4096, 1)
# weights = torch.load('/content/drive/My Drive/Colab Notebooks/model weights/vggface_vgg_3convs.pl')
# model.load_dict_state(weights)
# model.load_state_dict(weights)
# model.Module
# model.load_weights()
print(model)

if torch.cuda.is_available():
  print('using device: cuda')
else:
    print('using device: cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("done")
# print(feature_extractor)

VGG_16(
  (conv_1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_4_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_4_3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_5_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_5_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_5_3): 

In [10]:
win_set = []
los_set = []
with torch.no_grad():
        for data in extraction_loader_win:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs = data
            inputs = inputs.to(device)
            l25, l27, l29 = model(inputs)
            # print(l25.shape)
            m = nn.MaxPool2d(14, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            win_set.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), True])
            # break
print("HALWAY")           
with torch.no_grad():
        for data in extraction_loader_los:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs = data
            inputs = inputs.to(device)
            l25, l27, l29 = model(inputs)
            m = nn.MaxPool2d(14, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            los_set.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), False])


HALWAY


In [11]:
win_df = pd.DataFrame(win_set)
win_df.to_pickle('/content/drive/My Drive/Colab Notebooks/NF_vgg_vggface_features_win_set_train.pl')
los_df = pd.DataFrame(los_set)
los_df.to_pickle('/content/drive/My Drive/Colab Notebooks/NF_vgg_vggface_features_los_set_train.pl')

In [12]:
win_set_test = []
los_set_test = []
with torch.no_grad():
        for data in extraction_loader_win_test:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs = data
            inputs = inputs.to(device)
            l25, l27, l29 = model(inputs)

            m = nn.MaxPool2d(14, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            win_set_test.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), True])
with torch.no_grad():
        for data in extraction_loader_los_test:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs = data
            inputs = inputs.to(device)
            l25, l27, l29 = model(inputs)

            m = nn.MaxPool2d(14, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            los_set_test.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), False])
  

In [13]:
win_df_test = pd.DataFrame(win_set_test)
win_df_test.to_pickle('/content/drive/My Drive/Colab Notebooks/NF_vgg_vggface_features_win_set_test.pl')
los_df_test = pd.DataFrame(los_set_test)
los_df_test.to_pickle('/content/drive/My Drive/Colab Notebooks/NF_vgg_vggface_features_los_set_test.pl')

In [14]:
win_set_val = []
los_set_val = []
with torch.no_grad():
        for data in extraction_loader_win_val:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs = data
            inputs = inputs.to(device)
            l25, l27, l29 = model(inputs)

            m = nn.MaxPool2d(14, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            win_set_val.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), True])
print("HALFWAY")           
with torch.no_grad():
        for data in extraction_loader_los_val:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs = data
            inputs = inputs.to(device)
            l25, l27, l29 = model(inputs)

            m = nn.MaxPool2d(14, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            los_set_val.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), False])
  

HALFWAY


In [15]:
win_df_val = pd.DataFrame(win_set_val)
win_df_val.to_pickle('/content/drive/My Drive/Colab Notebooks/NF_vgg_vggface_features_win_set_val.pl')
los_df_val = pd.DataFrame(los_set_val)
los_df_val.to_pickle('/content/drive/My Drive/Colab Notebooks/NF_vgg_vggface_features_los_set_val.pl')

In [16]:
train_features = []
test_features = []
val_features = []

with torch.no_grad():
        for data in train_loader:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs, label = data
            inputs = inputs.to(device)
            l25, l27, l29 = model(inputs)
            m = nn.MaxPool2d(14, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            train_features.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), label])
       
with torch.no_grad():
        for data in test_loader:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs, label = data
            inputs = inputs.to(device)
            l25, l27, l29 = model(inputs)
            m = nn.MaxPool2d(14, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            test_features.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), label])
with torch.no_grad():
        for data in val_loader:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            inputs, label = data
            inputs = inputs.to(device)
            l25, l27, l29 = model(inputs)
            m = nn.MaxPool2d(14, stride=1)
            l29 = m(l29)
            l27 = m(l27)
            l25 = m(l25)
            val_features.append([l25.cpu().numpy(),l27.cpu().numpy(),l29.cpu().numpy(), label])
            


In [17]:
train_features = pd.DataFrame(train_features)
train_features.to_pickle('/content/drive/My Drive/Colab Notebooks/NF_vgg16_vggface_features_train.pl')
test_features = pd.DataFrame(test_features)
test_features.to_pickle('/content/drive/My Drive/Colab Notebooks/NF_vgg16_vggface_features_test.pl')
val_features = pd.DataFrame(val_features)
val_features.to_pickle('/content/drive/My Drive/Colab Notebooks/NF_vgg16_vggface_features_val.pl')

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
